# Juliaで固有値問題を精度保証付き数値計算したい


## 固有値問題とは

固有値問題は与えられた行列の固有値および固有ベクトルを求めること。つまり、
$
\newcommand{\C}{\mathbb{C}}
\newcommand{\IC}{\mathbb{IC}}
\newcommand{\bX}{\bar{X}}
$

$$
Ax=\lambda x,\quad A\in \C^{n\times n}
$$

を満たす$\lambda\in \C$, $x\in \C^{n\times n}$ ($x\neq 0$) を求める問題を指す。

Juliaで固有値と固有ベクトルを求める際には、`LinearAlgebra`パッケージに含まれている関数`eigen`・`eigvals`・`eigvecs`で求めることができる。

|関数| 役割 |
|:-----------|:----------|
| eigen()    | 固有値・固有ベクトル両方  |
| eigvals()    | 固有値  |
| eigvecs()  | 固有ベクトル |

多分、Lapackの何かを呼び足してるはず。

In [1]:
using LinearAlgebra
val, vec  = eigen([1 2; 2 1])

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -1.0
  3.0
vectors:
2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [ ]:
val#固有値

In [ ]:
vec#固有ベクトル

In [ ]:
eigvals([1 2; 2 1])#固有値

In [ ]:
eigvecs([1 2; 2 1])#固有ベクトル

## 密行列の固有値問題に対する精度保証

密行列とは、行列内のほとんどの要素が0でない行列のことを指す。ここでは、行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ の全ての固有値に対する精度保証アルゴリズムを考える。 

### 実装したいアルゴリズム
(1)
ある行列$A$の全ての近似固有ベクトル$\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

(2)
$A\cdot\bX$ の包含 $C\in\IC^{n\times n}$ を求める。

(3)
区間連立1次方程式 $\bX G=C$ の解集合の包含 $G\supset\bX^{-1}\cdot C$ を求める。

(4)
$G\in\IC^{n\times n}$ に対してゲルシュゴリンの定理を適用することを考える。

#### (1)行列$A$の全ての近似固有ベクトル$\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ が対角化可能であるとするとき、$\lambda_i$ に対する固有ベクトル$x^{(i)}\in\C^{n}$ $(i=1,2,\cdots,n)$ を並べた行列

$$
X :=[x^{(1)},x^{(2)},\cdots,x^{(n)}]\in\C^{n\times n}
$$

によって、$A$ は

$$
X^{-1}AX = D = \rm{diag}(\lambda_1,\lambda_2,\cdots,\lambda_n)
$$

と対角化される。

In [2]:
using LinearAlgebra

n =100;
A = randn(n,n);

λ, X = eigen(A);

In [ ]:
λ #固有値

In [ ]:
X #固有ベクトル

しかし、実際のところ上記の数値計算で得られる固有ベクトル $X$ は近似の値 $\bX\in \C^{n\times n}$ (以下、近似固有ベクトルという) である。しかし、近似固有ベクトルを並べた行列を使って、対角化したものについては、対角優位性が期待できる。この行列を $G$ とすると、

$$
G := \bX^{-1}A\bX
$$

となり、行列 $G$ の対角成分は行列 $A$ の固有値の近似となる。

また、対角優位性とは、ある行列$A=(a_{ij})$について、

$$
\left|a_{i i}\right|>\sum_{{j} \colon j \neq i}\left|a_{i j}\right| \quad(i=1,2, \ldots, n)
$$

であること。つまり、行列のどの対角成分の絶対値も、それと同じ列にある非対角成分の和よりも大きい時。

In [3]:
G = inv(X)*A*X; #G
norm(diag(G) - λ,Inf) # Gの対角成分は固有値λにほぼ一致する

4.796284432741295e-14

#### (2)$A\cdot\bX$の包含$C\in\IC^{n\times n}$を求める。

``方法1``各演算を区間演算に変更した方式(参考：[interval_dot-mul.ipynb](https://www.risk.tsukuba.ac.jp/~takitoshi/tutorial/interval_dot-mul.html "タイトル"))

In [4]:
using IntervalArithmetic, BenchmarkTools

iA = map(Interval, A);
iX = map(Interval, X);

@btime C1 = $iA * $iX;

  83.171 ms (8 allocations: 312.91 KiB)


In [5]:
@show max(maximum(radius.(real(C1[:]))),maximum(radius.(imag(C1[:]))))

LoadError: UndefVarError: C1 not defined

``方法2``BLASを使う。int_num_linalg.jlの呼び出し。この方が早い。

In [6]:
using BenchmarkTools
include("int_num_linalg.jl"); # int_mulを使用する
# @time C1 = mm_comp_interval_alt(iA,iX);
@time C = int_mul(A,X); # A, X: complex matrices (not interval)

  1.237924 seconds (4.64 M allocations: 222.128 MiB, 3.57% gc time)


In [7]:
max(maximum(radius.(real(C[:]))),maximum(radius.(imag(C[:]))))

9.103828801926284e-14

In [8]:
real(C)

100×100 Array{Interval{Float64},2}:
 [-0.957696, -0.957695]    …  [-0.261251, -0.26125]
  [0.583712, 0.583713]        [-0.890055, -0.890054]
 [-2.00467, -2.00466]          [0.974361, 0.974362]
 [-0.321647, -0.321646]       [-0.719869, -0.719868]
  [0.1279, 0.127901]          [-0.129917, -0.129916]
 [-0.972016, -0.972015]    …   [0.197544, 0.197545]
  [0.401453, 0.401454]         [1.02867, 1.02868]
  [1.07488, 1.07489]           [0.327513, 0.327514]
  [1.08726, 1.08727]           [1.38159, 1.3816]
  [1.32679, 1.3268]           [-0.533184, -0.533183]
 [-1.18749, -1.18748]      …  [-0.272931, -0.27293]
  [1.68066, 1.68067]           [0.938251, 0.938252]
  [0.351804, 0.351805]        [-0.0587759, -0.0587758]
   ⋮                       ⋱  
 [-0.517929, -0.517928]        [0.392765, 0.392766]
  [0.516897, 0.516898]        [-0.44515, -0.445149]
 [-0.732545, -0.732544]    …  [-0.248713, -0.248712]
 [-1.07352, -1.07351]         [-0.669768, -0.669767]
 [-0.712745, -0.712744]       [-0.771838, -0.

In [9]:
imag(C)

100×100 Array{Interval{Float64},2}:
 [-2.22508e-308, 2.22508e-308]  …   [0.327299, 0.3273]
 [-2.22508e-308, 2.22508e-308]      [0.437695, 0.437696]
 [-2.22508e-308, 2.22508e-308]      [0.130328, 0.130329]
 [-2.22508e-308, 2.22508e-308]     [-1.5532, -1.55319]
 [-2.22508e-308, 2.22508e-308]     [-0.058642, -0.0586419]
 [-2.22508e-308, 2.22508e-308]  …  [-0.160517, -0.160516]
 [-2.22508e-308, 2.22508e-308]      [0.568468, 0.568469]
 [-2.22508e-308, 2.22508e-308]     [-0.994355, -0.994354]
 [-2.22508e-308, 2.22508e-308]      [0.703024, 0.703025]
 [-2.22508e-308, 2.22508e-308]     [-0.844976, -0.844975]
 [-2.22508e-308, 2.22508e-308]  …  [-1.11569, -1.11568]
 [-2.22508e-308, 2.22508e-308]      [0.536674, 0.536675]
 [-2.22508e-308, 2.22508e-308]     [-0.308824, -0.308823]
   ⋮                            ⋱  
 [-2.22508e-308, 2.22508e-308]      [0.325833, 0.325834]
 [-2.22508e-308, 2.22508e-308]      [0.454863, 0.454864]
 [-2.22508e-308, 2.22508e-308]  …  [-0.425711, -0.42571]
 [-2.22508e-308

#### 3)区間連立1次方程式 $\hat XG=C$ の解集合の包含 $G\supset\hat X^{-1}\cdot C$ を求める。

In [10]:
function verifylss_ib(A,ib) # verify the solution element-wisely
    b = mid.(real(ib)) + mid.(imag(ib))*im;
    x̄ = A\b;
    n = length(x̄);
    R = inv(A);
    #########
    #C_mid, C_rad = mm_ufp(R,A);
    G = Matrix{Float64}(I, n, n) - int_mul(R,A);
    α = opnorm(G,Inf)# Interval arithmetic
    #########
    if α < 1
        x̄ = map(Interval,x̄);
        r = A*x̄ - ib; # Interval arithmetic
        Rr = R*r;
        err = abs.(Rr) + supremum(norm(Rr,Inf))/(1-α)*(abs.(G)*ones(n)); # Interval arithmetic
    else
        println("Oh my way, verification is failed...")
        err = nan;
    end
    return x = (real(x̄) .± supremum.(err)) + im*(imag(x̄) .± supremum.(err))
end

verifylss_ib (generic function with 1 method)

In [11]:
G = similar(C);
for i = 1:n
    G[:,i] = verifylss_ib(X,C[:,i]);
end
real(G)

100×100 Array{Interval{Float64},2}:
 [-10.0668, -10.0667]          …  [-9.33731e-13, 9.2166e-13]
  [-1.92634e-12, 1.92119e-12]     [-2.67348e-12, 2.68561e-12]
  [-1.92634e-12, 1.92119e-12]     [-2.65954e-12, 2.68244e-12]
  [-1.58058e-12, 1.57989e-12]     [-2.15458e-12, 2.14676e-12]
  [-1.58058e-12, 1.57989e-12]     [-2.14928e-12, 2.15408e-12]
  [-2.07963e-12, 2.07398e-12]  …  [-2.13588e-12, 2.11618e-12]
  [-1.14231e-12, 1.14284e-12]     [-1.58972e-12, 1.58797e-12]
  [-1.14231e-12, 1.14284e-12]     [-1.58938e-12, 1.57593e-12]
  [-1.42994e-12, 1.42444e-12]     [-1.94004e-12, 1.95977e-12]
  [-1.42994e-12, 1.42444e-12]     [-1.9593e-12, 1.93527e-12]
  [-1.16847e-12, 1.16445e-12]  …  [-1.62268e-12, 1.65134e-12]
  [-1.16847e-12, 1.16445e-12]     [-1.65479e-12, 1.61545e-12]
  [-1.79504e-11, 1.79445e-11]     [-1.86165e-11, 1.86729e-11]
    ⋮                          ⋱  
  [-3.34569e-12, 3.33806e-12]     [-4.05122e-12, 4.02299e-12]
  [-2.26444e-12, 2.2571e-12]      [-3.16517e-12, 3.20641e-12]
 

In [12]:
[real(diag(G)), λ]

2-element Array{Array{T,1} where T,1}:
 Interval{Float64}[[-10.0668, -10.0667], [-9.24817, -9.24816], [-9.24817, -9.24816], [-8.29201, -8.292], [-8.29201, -8.292], [-8.2613, -8.26129], [-7.95267, -7.95266], [-7.95267, -7.95266], [-7.26121, -7.2612], [-7.26121, -7.2612]  …  [6.1344, 6.13441], [7.41688, 7.41689], [7.61274, 7.61275], [7.61274, 7.61275], [7.64689, 7.6469], [7.64689, 7.6469], [7.88647, 7.88648], [7.88647, 7.88648], [9.07484, 9.07485], [9.07484, 9.07485]]
 Complex{Float64}[-10.066707272334748 + 0.0im, -9.248167089231517 - 1.0456274909042635im, -9.248167089231517 + 1.0456274909042635im, -8.292007068922306 - 2.4140926260253006im, -8.292007068922306 + 2.4140926260253006im, -8.261290538048907 + 0.0im, -7.9526610078135604 - 3.5643946497725816im, -7.9526610078135604 + 3.5643946497725816im, -7.261205663364121 - 1.5766450345163427im, -7.261205663364121 + 1.5766450345163427im  …  6.1344070486837 + 4.8702745388511355im, 7.416886103712434 + 0.0im, 7.612741584002496 - 1.9093896548159726

In [13]:
C=real(C)+imag(C)
C[1,1]

[-0.957696, -0.957695]

#### 4)$G\in\IC^{n\times n}$に対してゲルシュゴリンの定理を適用することを考える。


## ゲルシュゴリンの包含定理
行列 $A=(A_{ij})\in \C^{n\times n}$について、$A$の全ての固有値$\lambda_i$は、$\bigcup_{1\le i \le n}  U_i$の内部に存在する。

$$
A = \bigcup_{1\le i \le n}  U_i
$$

ただし、

$$
U_i = \{z\in\mathbb{C}:|z-a_{ii}|\le \sum_{j\neq i}|a_{ij}|\}
$$

である。

円盤領域$U_i$をゲルシュゴリン円板といい、Aが強い優対角性を持つとき、Aの対角成分がAの固有値の良い近似となる。

今回の場合、$G=(g_{ij})$の対角成分$[g_{ii}]$を$c_i$、半径を$\rm{rad(g_{ii})}$とすると、

$$
\begin{array}{ll}
\mbox{ゲルシュゴリン円の中心:}&c_i\\
\mbox{ゲルシュゴリン円の半径:}&r_i=\sum_{j\neq i}\rm{mag}(g_{ij})+\rm{rad}(g_{ii})\\
\end{array}
$$

となる。

In [29]:
for i = 1:n
    mr = mag.(real(G[i,:]))
    mi = mag.(imag(G[i,:]))
    magri = mr + mi
    rad = radius.(real(diag(G)[i])) + radius.(imag(diag(G)[i]))
    r = mr + mi + rad
end

LoadError: MethodError: no method matching +(::Array{Float64,1}, ::Float64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:538
  +(!Matched::IntervalArithmetic.IntervalRounding{:none}, ::T, !Matched::T, !Matched::RoundingMode{:Down}) where T<:AbstractFloat at /Users/K/.julia/packages/IntervalArithmetic/sRFlx/src/intervals/rounding.jl:180
  +(!Matched::IntervalArithmetic.IntervalRounding{:none}, ::T, !Matched::T, !Matched::RoundingMode{:Up}) where T<:AbstractFloat at /Users/K/.julia/packages/IntervalArithmetic/sRFlx/src/intervals/rounding.jl:180
  ...

In [17]:
mag.(real(G[1,:]))-mag.(real(diag(G)[1]))

LoadError: MethodError: no method matching -(::Array{Float64,1}, ::Float64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar
Closest candidates are:
  -(!Matched::IntervalArithmetic.IntervalRounding{:none}, ::T, !Matched::T, !Matched::RoundingMode{:Down}) where T<:AbstractFloat at /Users/K/.julia/packages/IntervalArithmetic/sRFlx/src/intervals/rounding.jl:180
  -(!Matched::IntervalArithmetic.IntervalRounding{:none}, ::T, !Matched::T, !Matched::RoundingMode{:Up}) where T<:AbstractFloat at /Users/K/.julia/packages/IntervalArithmetic/sRFlx/src/intervals/rounding.jl:180
  -(!Matched::Missing, ::Number) at missing.jl:115
  ...

In [ ]:
using Plots
plot!(index, max_rad, yscale=:log10,
    xlabel       ="Real",                #X軸のラベル
    ylabel       ="Imaginary",   #Y軸のラベル
    xlims        =(0,41),             #X軸の範囲
    title="gershgorin",       #タイトル
    linewidth    =2,                  #線幅
    seriestype = :scatter,            #点プロットに　
    size         =(400,300),          #プロットのサイズ
)

In [ ]:
for i in 1:n
    m = sum(mag.(imag(G[i,:])))-mag.(imag(diag(G)[i]))
    maggi =m +magg
end

In [ ]:
maggi = sum(mag.(imag(G[:])))-sum(mag.(imag(diag(G)[:])))

In [ ]:
magg = maggr + maggi

In [ ]:
radg = sum(radius.(real(diag(G)[:]))) + sum(radius.(imag(diag(G)[:])))

In [ ]:
mag.(real(diag(G)[1]))

In [ ]:
using IntervalArithmetic
function gershgorin(G)
    g=diag(G)
    center=[real(g),imag(g)]
    ##real
    for i =1:n
        re = real(g)
        radr[i]=radius(re[i])
    ##imag
     for j =1:n
        im = imag(g)
        radim[j]=radius(im[j])
end

対角優位
http://www.math.ritsumei.ac.jp/~yasutomi/jugyo/Numerical_Analysis/note5.pdf

計算途中がでる
http://www.cas.cmc.osaka-u.ac.jp/~paoon/misc/julia/post/cg/


http://infsup.jp/saito/materials/na19exer.pdf